In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.compose import TransformedTargetRegressor

from catboost import CatBoostRegressor


train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

train_labels = train_set["SalePrice"]
train_set = train_set.drop("SalePrice", axis=1)

num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
cat_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"), OrdinalEncoder())

auto_preprocess = make_column_transformer(
    (num_pipeline, make_column_selector(dtype_include=np.number)),
    (cat_pipeline, make_column_selector(dtype_include=object)),
)

log_level_regressor = TransformedTargetRegressor(
    CatBoostRegressor(random_state=42, silent=True, allow_writing_files=False),
    func=np.log,
    inverse_func=np.exp,
)

final_model = make_pipeline(
    auto_preprocess,
    log_level_regressor,
)

final_model.fit(train_set, train_labels)
# Dummy only because there isn't any analysis. CatBoost is an excellent model.
dummy_results = final_model.predict(test_set)

df = pd.DataFrame({"Id": test_set["Id"], "SalePrice": dummy_results})
df.set_index("Id").to_csv("dummy_submission.csv")
df.head()

NameError: name 'train_path' is not defined